In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/LLM

/content/drive/My Drive/LLM


In [ ]:
%ls Documents

Documents/


In [ ]:
import os
os.getcwd()

'/content/drive/MyDrive/LLM'

In [ ]:
!ls

Documents  LLM_2_RAG.ipynb  requirements.txt


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Pinecone
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [ ]:
# setting Hugging face key
from langchain.llms import HuggingFaceHub
import os
from getpass import getpass
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("HF token")

HF token··········


In [ ]:
# Loads PDF files available in a directory with pypdf
def load_docs(directory):
  loader = PyPDFDirectoryLoader(directory)
  documents = loader.load()
  return documents

In [ ]:
directory = 'Documents/'
documents = load_docs(directory)
len(documents)

36

In [ ]:
documents[:3]

[Document(page_content='Antimicrobial Stewardship Program  \n- 1 - \n Bronchitis  \n \n \nWhat is Acute Bronchitis?  \nBronchitis occurs when the airways of the lungs swell and produce mucus. \nThat’s what makes you cough. Acute bronchitis, often called a “chest cold,” is the most common type of bronchitis. The symptoms last less than 3 weeks. \nAntibiotics are not needed to treat acute bronchitis. Using antibiotics when not \nneeded could do more harm than good.  \n \nIf you’re a healthy person without underlying heart or lung problems or a weakened immune system, this information is for you.  \n \nWhat are the s ymptoms of \nAcute Bronchitis ? \n• Coughing with or without \nmucus production  \nYou may also experience:   \n• Soreness in the chest \n• Fatigue (feeling tired)   \n• Mild headache  \n• Mild body aches  \n• Watery eyes  \n• Sore throat  \n \n \n \n \n  \nBronchitis occurs when the airways of the lungs \nswell and produce mucus. That’s what makes you cough.  \n', metadata={

In [ ]:
def split_docs(documents, chunk_size=512, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap,
                                                 separators=["\n\n", "\n", " ", ""])
  docs = text_splitter.split_documents(documents)
  return docs

In [ ]:
docs = split_docs(documents)
print(len(docs))

337


In [ ]:
docs[:10]

[Document(page_content='Antimicrobial Stewardship Program  \n- 1 - \n Bronchitis  \n \n \nWhat is Acute Bronchitis?  \nBronchitis occurs when the airways of the lungs swell and produce mucus. \nThat’s what makes you cough. Acute bronchitis, often called a “chest cold,” is the most common type of bronchitis. The symptoms last less than 3 weeks. \nAntibiotics are not needed to treat acute bronchitis. Using antibiotics when not \nneeded could do more harm than good.', metadata={'source': 'Documents/Documents/Documents/bronchitis.pdf', 'page': 0}),
 Document(page_content='If you’re a healthy person without underlying heart or lung problems or a weakened immune system, this information is for you.  \n \nWhat are the s ymptoms of \nAcute Bronchitis ? \n• Coughing with or without \nmucus production  \nYou may also experience:   \n• Soreness in the chest \n• Fatigue (feeling tired)   \n• Mild headache  \n• Mild body aches  \n• Watery eyes  \n• Sore throat  \n \n \n \n \n  \nBronchitis occurs w

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query_result = embeddings.embed_query("Hello Buddy")
len(query_result)

384

In [ ]:
query_result

[-0.0697883665561676,
 0.05420626699924469,
 0.07814779132604599,
 0.03390125930309296,
 0.024947451427578926,
 -0.09673727303743362,
 0.059523165225982666,
 0.05897817388176918,
 -0.01789672300219536,
 -0.023178907111287117,
 -0.019000250846147537,
 0.000596922414842993,
 0.02466605417430401,
 -0.07030829787254333,
 -0.0075225443579256535,
 0.010224469006061554,
 -0.011180826462805271,
 -0.02124853804707527,
 -0.03859451785683632,
 0.02655033767223358,
 -0.06505249440670013,
 0.06500022113323212,
 0.009431741200387478,
 -0.06271225214004517,
 -0.023625388741493225,
 -0.030638137832283974,
 0.05996118485927582,
 0.07367482036352158,
 -0.03286784142255783,
 -0.02606104500591755,
 -0.006967148743569851,
 0.030617928132414818,
 0.059396713972091675,
 0.0014720155159011483,
 0.012021704576909542,
 0.02829374186694622,
 -0.05922526866197586,
 -0.07919744402170181,
 0.048963677138090134,
 0.023090053349733353,
 0.055362846702337265,
 -0.026251360774040222,
 -0.017321137711405754,
 0.00551118

In [ ]:
from pinecone import Pinecone as PineconeClient
from langchain_community.vectorstores import Pinecone

os.environ["PINECONE_API_KEY"] = getpass("PINECONE_API_KEY:")

PINECONE_API_KEY:··········


In [ ]:
PineconeClient(api_key="PINECONE_API_KEY", environment="gcp-starter")
index_name = "myindex"
index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [ ]:
def get_similar_docs(query, k=4):
    similar_docs = index.similarity_search(query, k=k)
    return similar_docs

In [ ]:
rd = get_similar_docs("what is asthma")
rd

[Document(page_content='Introduction to Asthma \nChildhood asthma can be controlled through educatio n and treatment. Working as a team, school \npersonnel, healthcare providers and parents can hel p children with asthma participate fully in school,  \nsports and home activities.  \nWhat is Asthma?  \nAsthma is one of the most common chronic diseases o f \nchildhood, affecting more than 6 million children. \nAsthma is a chronic inflammatory lung disease that can \ncause repeated episodes of cough, wheezing and brea thing', metadata={'page': 0.0, 'source': 'Documents/Documents/Documents/asthma.pdf'}),
 Document(page_content='Introduction to Asthma \nChildhood asthma can be controlled through educatio n and treatment. Working as a team, school \npersonnel, healthcare providers and parents can hel p children with asthma participate fully in school,  \nsports and home activities.  \nWhat is Asthma?  \nAsthma is one of the most common chronic diseases o f \nchildhood, affecting more than 6 

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

In [ ]:
from langchain.llms import HuggingFaceEndpoint
import os
from getpass import getpass

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("HF token")

HF token··········


In [ ]:
llm=HuggingFaceHub(repo_id="google/flan-t5-xxl",
                   model_kwargs={"temperature":0.1,"max_new_tokens":500})
llm

HuggingFaceHub(client=<InferenceClient(model='google/flan-t5-xxl', timeout=None)>, repo_id='google/flan-t5-xxl', task='text2text-generation', model_kwargs={'temperature': 0.1, 'max_new_tokens': 500})

In [ ]:
chain = load_qa_chain(llm, chain_type="stuff", verbose=False)

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    Based only on the summaries provided below, answer the given question.
    If you don't know the answer, respond with "I don't know".
    Do not generate new questions.
    Do not make up your own answer from outside the source material.
    Do not write anything after the answer.

    Source material: {context}

    Question: {question}
    Answer:
    """
)

chain = load_qa_chain(
    llm,
    chain_type="stuff",
    prompt=prompt,
)

In [ ]:
def get_answer(query):
  relevant_docs = get_similar_docs(query)
  response = chain.run(input_documents=relevant_docs, question=query)
  return response

In [ ]:
our_query = "What is Asthma?"
answer = get_answer(our_query)
print(answer)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Asthma is one of the most common chronic diseases o f childhood, affecting more than 6 million children. Asthma is a chronic inflammatory lung disease that can cause repeated episodes of cough, wheezing and brea thing


In [ ]:
our_query = "who is Michal Jackson?"
answer = get_answer(our_query)
print(answer)

I don't know


In [ ]:
our_query = "What are the triggers for Asthma?"
answer = get_answer(our_query)
print(answer)

Allergic reactions to environmental allergens such as pollens, molds, dust mite or animal dander • Colds and viral respiratory infections • Exercise • Changes in weather, exposure to cold air or sudden temperature change • Irritants such as tobacco smoke, air pollution, pa ints and cleaning agents • Strong odors and/or perfumes Exercise-Induced Bronchoconstriction, (EIB) For some people, asthma symptoms may only occur dur ing exercise. A history of cough, shortness of


In [ ]:
our_query = "What is the recommended treatement for acute bronchitis?"
answer = get_answer(our_query)
print(answer)

Acute bronchitis almost always gets better on its own—without antibiotics. Using antibiotics when they aren’t needed can do more harm than good. Unintended consequences of antibiotics include side effects, like rash and diarrhea, as well as more se rious consequences, such as an increased risk for an


In [ ]:
our_query = "List some cyber attacks that took place during the Covid-19 pandemic."
answer = get_answer(our_query)
print(answer)

scams and phishing, malware, and dis- tributed denial-of-service (DDoS).


In [ ]:
our_query = our_query = "Which classifier gave the best performance for thyroid disease treatment prediction?"
answer = get_answer(our_query)
print(answer)


I don't know
